In [2]:
import os
import numpy as np
import pandas as pd 
import json
from PIL import Image


# Image Classification

### ArtDL

In [4]:
(df.iloc[:,1:-1].sum(axis=1) == 0).sum()

12833

In [6]:
columns_rename = {'11H(ANTONY OF PADUA)': 'Anthony of Padua',
                  '11H(FRANCIS)': 'Francis of Assisi',
                  '11H(JEROME)': 'Jerome',
                  '11H(JOHN THE BAPTIST)': 'John the Baptist',
                  '11HH(MARY MAGDALENE)': 'Mary Magdalene',
                  '11H(PAUL)': 'Paul',
                  '11H(PETER)': 'Peter',
                  '11H(DOMINIC)': 'Saint Dominic',
                  '11H(SEBASTIAN)': 'Saint Sebastian',
                  '11F(MARY)': 'Virgin Mary',
                  }

In [7]:
cols_to_drop = [col for col in df.columns[1:-1] if col not in columns_rename]
cols_to_drop

['11H(ANTONY ABBOT)',
 '11H(AUGUSTINE)',
 '11H(JOHN)',
 '11H(JOSEPH)',
 '11H(STEPHEN)',
 '11HH(BARBARA)',
 '11HH(CATHERINE)',
 'John Baptist - Child',
 'John Baptist - Dead']

In [8]:
df.drop(cols_to_drop, axis=1, inplace=True)

In [11]:
filt_ = df.iloc[:,1:-1].sum(axis=1) == 1
df = df.loc[filt_].copy()

In [14]:
df.rename(columns = columns_rename, inplace=True)

In [17]:
df["class"] = ''
for col in df.columns[1:-2]:
    print(col)
    filt_ = df[col] == 1
    df.loc[filt_, "class"] = col

Virgin Mary
Anthony of Padua
Saint Dominic
Francis of Assisi
Jerome
John the Baptist
Paul
Peter
Saint Sebastian
Mary Magdalene


In [25]:
df_train = df.loc[df["set"]=="train"].copy()
df_test = df.loc[df["set"]=="test"].copy()
df_val = df.loc[df["set"]=="val"].copy()

In [29]:
image_files = os.listdir("/home/shashank/tensorflow_datasets/downloads/art_dl/ArtDL/JPEGImages")

In [30]:
train_missing = []
for img in df_train["item"]:
    if img+".jpg" not in image_files:
        train_missing.append(img)
        
test_missing = []
for img in df_test["item"]:
    if img+".jpg" not in image_files:
        test_missing.append(img)
        
val_missing = []
for img in df_val["item"]:
    if img+".jpg" not in image_files:
        val_missing.append(img)
        
train_missing, test_missing, val_missing

([], [], [])

In [31]:
images_info = {}
images_info["categories"] = []
i = 0
for value in columns_rename.values():
    x = {}
    x["id"] = i
    x["name"] = value
    images_info["categories"].append(x)
    i+=1

images_info

{'categories': [{'id': 0, 'name': 'Anthony of Padua'},
  {'id': 1, 'name': 'Francis of Assisi'},
  {'id': 2, 'name': 'Jerome'},
  {'id': 3, 'name': 'John the Baptist'},
  {'id': 4, 'name': 'Mary Magdalene'},
  {'id': 5, 'name': 'Paul'},
  {'id': 6, 'name': 'Peter'},
  {'id': 7, 'name': 'Saint Dominic'},
  {'id': 8, 'name': 'Saint Sebastian'},
  {'id': 9, 'name': 'Virgin Mary'}]}

In [32]:
path = "/home/shashank/tensorflow_datasets/downloads/art_dl/ArtDL/JPEGImages"
map = {}
for x_dict in images_info["categories"]:
    map[x_dict["name"]] = x_dict["id"]
map

{'Anthony of Padua': 0,
 'Francis of Assisi': 1,
 'Jerome': 2,
 'John the Baptist': 3,
 'Mary Magdalene': 4,
 'Paul': 5,
 'Peter': 6,
 'Saint Dominic': 7,
 'Saint Sebastian': 8,
 'Virgin Mary': 9}

In [33]:
train_images_info = images_info.copy()
train_images_info["images"] = []

i=0
for (img, label) in df_train[["item", "class"]].values:
    img_path = os.path.join(path, img+".jpg")
    x = {}
    x["label"] = map[label]
    x["file_name"] =  img_path
    img = Image.open(x["file_name"])
    img_array = np.array(img)
    x["height"] = img_array.shape[0]
    x["width"] = img_array.shape[1]
    x["id"] = i
    train_images_info["images"].append(x)
    i+=1
    

In [34]:
test_images_info = images_info.copy()
test_images_info["images"] = []

i=0
for (img, label) in df_test[["item", "class"]].values:
    img_path = os.path.join(path, img+".jpg")
    x = {}
    x["label"] = map[label]
    x["file_name"] =  os.path.join(img_path)
    img = Image.open(x["file_name"])
    img_array = np.array(img)
    x["height"] = img_array.shape[0]
    x["width"] = img_array.shape[1]
    x["id"] = i
    test_images_info["images"].append(x)
    i+=1
    

In [35]:
val_images_info = images_info.copy()
val_images_info["images"] = []

i=0
for (img, label) in df_val[["item", "class"]].values:
    img_path = os.path.join(path, img+".jpg")
    x = {}
    x["label"] = map[label]
    x["file_name"] =  os.path.join(img_path)
    img = Image.open(x["file_name"])
    img_array = np.array(img)
    x["height"] = img_array.shape[0]
    x["width"] = img_array.shape[1]
    x["id"] = i
    val_images_info["images"].append(x)
    i+=1
    

In [36]:
path = '/home/shashank/tensorflow_datasets/downloads/art_dl/ArtDL/'

In [37]:
import json 

with open(os.path.join(path, "test_images_info.json"), "w") as outfile: 
    json.dump(test_images_info, outfile)
    
with open(os.path.join(path, "train_images_info.json"), "w") as outfile: 
    json.dump(train_images_info, outfile)
    
with open(os.path.join(path, "val_images_info.json"), "w") as outfile: 
    json.dump(val_images_info, outfile)

### Skin Cancer


In [2]:
mapper = {"actinic_keratoses": 0,
          "basal_cell_carcinoma": 1,
          "benign_keratosis-like_lesions": 2,
          "dermatofibroma": 3,
          "melanocytic_Nevi": 4,
          "melanoma": 5,
          "vascular_lesions": 6,
          }

In [3]:
images_info = {}
images_info["categories"] = []
i = 0
for (key, val) in mapper.items():
    x = {}
    x["id"] = val
    x["name"] = key
    images_info["categories"].append(x)
    i+=1

images_info

{'categories': [{'id': 0, 'name': 'actinic_keratoses'},
  {'id': 1, 'name': 'basal_cell_carcinoma'},
  {'id': 2, 'name': 'benign_keratosis-like_lesions'},
  {'id': 3, 'name': 'dermatofibroma'},
  {'id': 4, 'name': 'melanocytic_Nevi'},
  {'id': 5, 'name': 'melanoma'},
  {'id': 6, 'name': 'vascular_lesions'}]}

In [ ]:

train_images_info = images_info.copy()
train_images_info["images"] = []
path = "/home/shashank/tensorflow_datasets/downloads/skin_cancer/train"
i=0

for category in mapper.keys():
    for file in os.listdir(os.path.join(path, category)):
        x = {}
        x["label"] = mapper[category]
        x["file_name"] =  os.path.join(path, category, file)
                
        img = Image.open(x["file_name"])
        img_array = np.array(img)
        x["height"] = img_array.shape[0]
        x["width"] = img_array.shape[1]
        x["id"] = i
        train_images_info["images"].append(x)
        i+=1

In [15]:
test_images_info = images_info.copy()
test_images_info["images"] = []
path = "/home/shashank/tensorflow_datasets/downloads/skin_cancer/test"
i=0

for category in mapper.keys():
    for file in os.listdir(os.path.join(path, category)):
        x = {}
        x["label"] = mapper[category]
        x["file_name"] =  os.path.join(path, category, file)
                
        img = Image.open(x["file_name"])
        img_array = np.array(img)
        x["height"] = img_array.shape[0]
        x["width"] = img_array.shape[1]
        x["id"] = i
        test_images_info["images"].append(x)
        i+=1

In [16]:
validation_images_info = images_info.copy()
validation_images_info["images"] = []
path = "/home/shashank/tensorflow_datasets/downloads/skin_cancer/validation"
i=0

for category in mapper.keys():
    for file in os.listdir(os.path.join(path, category)):
        x = {}
        x["label"] = mapper[category]
        x["file_name"] =  os.path.join(path, category, file)
                
        img = Image.open(x["file_name"])
        img_array = np.array(img)
        x["height"] = img_array.shape[0]
        x["width"] = img_array.shape[1]
        x["id"] = i
        validation_images_info["images"].append(x)
        i+=1

In [17]:
path = '/home/shashank/tensorflow_datasets/downloads/skin_cancer'

In [18]:
import json 
with open(os.path.join(path, "train_images_info.json"), "w") as outfile: 
    json.dump(train_images_info, outfile)

with open(os.path.join(path, "test_images_info.json"), "w") as outfile: 
    json.dump(test_images_info, outfile)
    
with open(os.path.join(path, "validation_images_info.json"), "w") as outfile: 
    json.dump(validation_images_info, outfile)